In [ ]:
%%HTML
<p id = 'heading1_p'> Symmetric Authentication – ATECC608-TFLXTLS </p>
<hr id = 'hline'> </hr> 
<p id = 'heading1_p' style='font-size: 20px; font-weight: bold;'>Introduction</p>
<div id='div_element'>
<p>Accessory/Disposable authentication is a process in which the accessory or the disposable client device generate MAC (Message Authentication Code) 
from a host device challenge. The MAC is verified (checkMAC) by the host device to authenticate the accessory/disposable device.</p>
<p>The purpose of authentication is to prevent cloning and counterfeiting and to ensure that an object is genuine and authorized to connect to a product.</p>

<p style='font-weight: bold;'>Implementation methods :</p>
<div id='div_element'>
<ul>
    <li><b>Symmetric Authentication - Diversified Key </b> - requires to integrate a Secure Element on the Device (Accessory / Disposable) and the Host side. Device Secure Element will be provisioned with a unique symmetric key (derived from a root symmetric key and the Secure Element serial number. Host Secure Element will be provisioned with the root symmetric key)</li>
    <li><b>Symmetric Authentication - Shared Key </b> - requires to integrate a Secure Element on the Device (Accessory / Disposable) and the Host side – both Secure Element will be provisioned with the same symmetric key.</li>
    <li><b>Symmetric Authentication - Stored Challenge/Response pairs for Small MCU (For resource constrained microcontrollers)</b> - requires a Secure Element on the Device (Accessory / Disposable) side only, leaving the possibility for Host cloning. Secure Element will be provisioned with a symmetric key, Host firmware will embed one or several challenge/response pair(s)</li>    
</ul>
</div>
<p>This use case describes how ECC608 can be configured as client accessory/disposable while a MCU acts as a host. The symmetric key in this case is stored on the MCU firmware for implementing 
the <b>Shared Key</b> version of the usecase. Ideally you would want to have a second ECC608 device connected to the HOST storing the symmetric key, this helps in securing the symmetric key.
The shared symmetric key is now protected and isolated from outside exposure (contract manufacturers, code bugs, users) and stored securely in the client ECC608 device.</p>
</div>

<button class="collapsible" style='font-size: 20px; font-weight: bold;'>Cryptographic Asset list & Use Case implementation</button>
<div class="content">
<div id='content_element'>
<hr id = 'h_content'> </hr> 
<p style="float: left; font-size: 9pt; text-align: center; width: 30%; margin-right: 1%;">
    <img src="assets/images/crypto-assets.png" alt="Cryptographic Assets" style="height: 35em; width: 100%"><b>Cryptographic Asset list</b></p>
<p style="float: left; font-size: 9pt; text-align: center; 20em; width: 40%; margin-right: 1%;">
    <img src="assets/images/usecase_steps.png" alt="Use Case Steps" style="height: 20em; width: 100%"><b>Use Case implementation</b></p>
<p style="clear: both;">
</div>
</div>

<p id = 'heading1_p' style='margin-top:40px; font-size: 20px; font-weight: bold;'>Use Case information</p>
<hr id = 'hline'> </hr> 
<div id='div_element'>
<p> The Transaction Diagram represent all steps to be performed to successfully implement the given Use Case, 
during development and after product delivery to the end-user. </p>
<p><strong> For the ease of the execution, when steps are executed from transaction diagram, these are emulated in the Host-PC 
rather than executed in the Host-Microcontroller/Processor from the evaluation kit.</strong> The MPLAB-X project/C Source code includes software code to be executed in your Host-Microcontroller/Processor 
in your embedded project. </p>

<p>Select your Evaluation kit and then click on each step to execute the Use Case. Click on Usecase Help button in 
transaction diagram for additional details.</p>
</div>

In [ ]:
import os
from tpds.usecase_diagram import ProtoBoard, UsecaseDiagram
from tpds.tp_utils.tp_settings import TPSettings
from tpds.proto_boards import get_board_path
from IPython.display import display

curr_path = os.path.abspath('')
os.sys.path.extend([os.path.join(curr_path, 'assets', 'python')])
import symmetric_authentication

tp_settings = TPSettings()
usecase_working_dir = os.path.join(tp_settings.get_base_folder(), 'spg_symm_auth')
os.makedirs(usecase_working_dir, exist_ok=True)
boards = ProtoBoard(max_width=400)
boards.add_board(
            'DM320118', 
            os.path.join(get_board_path(), 'DM320118', 'DM320118.png'), 
            os.path.join(get_board_path(), 'DM320118', 'DM320118.hex'),
            os.path.join(curr_path, 'firmware', 'ecc608_dm320118_symmauth_shared.X')
)
boards.add_board(
            'EBV-IoT Mchp Secure Shield', 
            os.path.join(get_board_path(), 'EBV-IoT_Mchp_Secure_Shield', 'EBV-IoT_Mchp_Secure_Shield.png'), 
            os.path.join(get_board_path(), 'EBV-IoT_Mchp_Secure_Shield', 'EBV-IoT_Mchp_Secure_Shield.hex'),
)
boards.render_boards(0)

accessory_auth = symmetric_authentication.SymmetricAuthentication(boards, symm_key_slot=0x05)
usecase_td = UsecaseDiagram(os.path.join(curr_path, 'assets', 'images', 'td-accessory-symmetric-auth.png'), working_dir=usecase_working_dir)
usecase_td.add_firmware(boards.get_firmware_project, tp_settings.get_mplab_paths().get('ide_path'))

generate_resources = usecase_td.add_script(237,68,35,30, accessory_auth.generate_resources)
initiate_auth = usecase_td.add_script(287,227,35,30, accessory_auth.initiate_accessory_auth, prereq_scripts = [generate_resources])
get_dev_mac = usecase_td.add_script(288,284,35,30, accessory_auth.get_mac_from_accessory_device, prereq_scripts = [initiate_auth])
compare_mac = usecase_td.add_script(290,350,35,30, accessory_auth.compare_host_mac_with_accessory_mac, prereq_scripts = [get_dev_mac])

usecase_td.add_image(825,201,140,25, os.path.join(curr_path, 'assets', 'images', 'c-snippet.PNG'), 'Code Snippet')
usecase_td.add_link(723,239,140,10,
    'https://github.com/MicrochipTech/cryptoauthlib/blob/af8187776cd3f3faf8bed412eaf6ff7221862e19/lib/basic/atca_basic_read.c#L116')
usecase_td.add_link(736,257,110,10,
    'https://github.com/MicrochipTech/cryptoauthlib/blob/af8187776cd3f3faf8bed412eaf6ff7221862e19/lib/basic/atca_basic_nonce.c#L174')
usecase_td.add_link(771,287,70,10,
    'https://github.com/MicrochipTech/cryptoauthlib/blob/af8187776cd3f3faf8bed412eaf6ff7221862e19/lib/basic/atca_basic_mac.c#L52')
usecase_td.add_link(753,339,100,15,
    'https://github.com/MicrochipTech/cryptoauthlib/blob/af8187776cd3f3faf8bed412eaf6ff7221862e19/lib/basic/atca_basic_checkmac.c#L49')
usecase_td.add_usecase_help(os.path.join(curr_path, 'assets', 'Symmetric Authentication.md'))

display(boards.canvas)
usecase_td.display_canvas()

In [ ]:
%%HTML
<button class="collapsible" style='font-size: 20px; font-weight: bold;'>Conclusion</button>
<div class="content">
<div id='content_element'>
<hr id = 'h_content'> </hr> 
<p>Please navigate back to the TrustFlex Secret Exchange page to finalize your Accessory symmetric Authentication Use Case.</p>
<ul><li>You will need to populate slot 5</li>
<li>Slot 5 (Symmetric Key) will be provisioned by Microchip</li>
</ul>
</div>
</div>